In [1]:
# !git clone https://github.com/wandb/tiny-ml.git

In [1]:
import tensorflow as tf
import wandb
from pathlib import Path
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import numpy as np
import sounddevice as sd
import wave
from pvrecorder import PvRecorder
import time 
import recorder
from utils.data_processing import augmenter,log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect
from utils.create_sweep import create_sweep

In [2]:
# some utilities that weve made in 
import recorder
from utils.data_processing import augmenter,log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect
from utils.create_sweep import create_sweep

In [3]:
# %load utils/data_processing.py

![](slides/tiny_ml_cover.svg)

![](slides/ml_morphology.svg)

![](slides/mlops_graph.svg)

![](slides/edge_devices.svg)


 <tr>
    <td> <img src="./slides/spark_micro_mod_pico_1.jpg" alt="Drawing" style="height: 100%;width:100%;" align="right"/> </td>
    <td> <img src="./slides/micro_mod_pico_2.jpg" alt="Drawing" style="height: 60%; width:100%" align="left"/> </td>
    </tr>


In [4]:
tqdm_args = dict(leave=True,position=0)
tqdm_colours = (f'#{r:02x}{g:02x}{b:02x}'for r,g,b in [(int(255*s), 3, int(255*s)) for s in np.linspace(1,0,100)])
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='Data')

In [5]:
recorder = PvRecorder(device_index=0, frame_length=512)
sound = []
recorder.start()
print('recording')
t_0 = time.time()
while time.time()-t_0<4:
    frame = recorder.read()
    sound.extend(frame)
recorder.stop()

recording


In [6]:
data = np.array(sound).astype(np.int16)
t_step = np.arange(0, len(data))
fig = px.line(x=t_step, y=data, title='a sound wave')
run.log({'sound_wave': fig})

### Showing a Run in Notebook

In [8]:
run

In [9]:
path = './data/yes/yes_record 102.wav'
run.log({'test_sound':wandb.Audio(path)})
run

# The basis of our signal processing is using Fast Fourier Transform (FFT)
> - [here](https://www.youtube.com/watch?v=spUNpyF58BY) is an amazing video on how FFT works. 
> - quickly go through getting from FFT to a Spectrogram. 
>  ![](./slides/200px-Fourier_in_his_coat_of_prefect.jpg)

In [10]:
fft = np.fft.fft(data).astype(np.int16)
t_steps = np.arange(0,len(fft))
fft = np.stack([fft,t_step]).T
df = pd.DataFrame(fft, columns = ['Faurrier_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Faurrier_Value', title='Fourrier Transform')
run.log({'Fouriere Transform':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_7942/660930182.py:1: ComplexWarning:

Casting complex values to real discards the imaginary part



In [11]:
run

## FFT can also be inverted, we can get sound back

In [12]:
fft = np.fft.fft(data)
roll = np.roll(fft,200)
ifft = np.fft.ifft(roll).astype(np.int16)
t_step = np.arange(len(ifft))
ifft = np.stack([ifft,t_step]).T
df = pd.DataFrame(ifft, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', 
              y='Frequecy_Value', 
              title='Inverse Fourrier Transform to Get sound Back')
run.log({'Inverset Fouriere':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_7942/1994990191.py:3: ComplexWarning:

Casting complex values to real discards the imaginary part



In [13]:
run

## Down Sample using FFT as an example of Signal Processing

In [14]:
# and examper of downsampling using FFT
roll = np.roll(fft,15)
ifft_ten = np.fft.ifft(roll)
ifft_ten = ((2**(16-4)) * ifft_ten/ifft_ten.max()).astype(np.int16)
ifft_ten = np.stack([ifft_ten,t_step]).T
df = pd.DataFrame(ifft_ten, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Frequecy_Value', title='Dowsampling using Fast Fourrier Transform (FFT)')
run.log({'Down_Sampled_Sound':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_7942/3482508054.py:4: ComplexWarning:

Casting complex values to real discards the imaginary part



In [15]:
run

In [16]:
run.finish()
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection ',
                 group='Data',
                 resume='must')

### ARM spectrogram using FFT 🦾


In [17]:
category = 'yes'
idx = 10
sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
sound = sound.astype(np.float32, order='F') / 32768.0
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
get_arm_spectrogram(sound)

array([[0.5     , 0.3125  , 0.125   , ..., 0.      , 0.      , 0.      ],
       [0.671875, 0.375   , 0.0625  , ..., 0.015625, 0.      , 0.      ],
       [0.78125 , 0.390625, 0.03125 , ..., 0.      , 0.      , 0.      ],
       ...,
       [1.109375, 0.53125 , 0.125   , ..., 0.015625, 0.      , 0.      ],
       [0.96875 , 0.375   , 0.03125 , ..., 0.015625, 0.      , 0.      ],
       [0.921875, 0.34375 , 0.0625  , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)

### Showing output of custom function optimized for arm processor

In [21]:
category = 'yes'
sound = read_wav(f'./data/{category}/yes_record 119.wav')
print(sound)
sound = sound.astype(np.float32, order='F') / 32768.0
# what we are using 
arm_spct = get_arm_spectrogram(sound)
# what were not using 
tf_spect = tf.signal.stft(sound, frame_length=512, frame_step=128).numpy().astype(np.float32)
fig = px.imshow(np.array(arm_spct))
run.log({f'spectrogram_{category}':fig})

[   0    0    0 ... -591 -643 -631]


/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_7942/3992365391.py:8: ComplexWarning:

Casting complex values to real discards the imaginary part



In [22]:
run.finish()


# Adding spectrogram plot to wandb table 🌌

# logging some media outside of a table to workspace
This is for:
- cheching what our network see in signal domain;
- to spot patterns;
- gain intuitions about process and raw signal data.

In [23]:
sample_rate = 16000
category = 'yes'
n_samples = 10
for idx in tqdm(range(1,n_samples), colour=next(tqdm_colours)):
    sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
    sound = sound.astype(np.float32, order='F') / 32768.0
    arm_spect = get_arm_spectrogram(sound)
    img = plot_spectrogram(arm_spect,sample_rate=sample_rate)
    img = wandb.Image(img)
    run.log({f'spectrogram_image{category}':img})

100%|█████████████████████████████████████████████| 9/9 [00:02<00:00,  3.69it/s]


In [27]:
run.finish()
run

In [28]:
!rm -rf datasets
artifact = wandb.Artifact(name='metadata', type='ESC-50-master')
artifact.add_reference('https://github.com/karoldvl/ESC-50/archive/master.zip')
artifact = run.use_artifact('tiny-ml/wake_word_detection/metadata:v0', type='ESC-50-master')
artifact_dir = artifact.download()
!mkdir datasets
!unzip ./artifacts/metadata:v0/master.zip -d /Users/fridadesigley/pico/tiny-ml/datasets/ 

KeyboardInterrupt: 

# 2. This Part covers pre training our model on ESC 50 dataset

### Reading our pre-traning data to pandas

In [29]:
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection', 
                 group='Data',
                 resume='must')
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
df = pd.read_csv(esc50_csv)
initial_data_table = wandb.Table(data=df, columns=list(df.columns))
run.log({'initial_table':initial_data_table})

Problem at: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_7942/1036862475.py 1 <module>


Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1108, in init
    run = wi.init()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 742, in init
    raise error
wandb.errors.CommError: Error communicating with wandb process, exiting...
For more info see: https://docs.wandb.ai/library/init#init-start-error


Exception: problem

### Here we are training on 1 second sound clips and our ESC 50 Dataset is 4 seconds per class

### testing our function 

In [ ]:
test_file = 'datasets/ESC-50-master/audio/1-100032-A-0.wav'
data = segment(fid=test_file,chunk=1)


### reading 1 second clips writing to a new files

In [ ]:
in_paranet = Path('./datasets/ESC-50-master/audio/')
out_data_dir = Path('ESC-50')
out_data_dir.mkdir(exist_ok=True)
all_out = [ ]
for idx, row in tqdm(df.iterrows()):
    out_dir = out_data_dir/row.category
    out_dir.mkdir(exist_ok=True)
    data, vals = segment(fid = str(in_paranet/row.filename),
       chunk=1)
    chans, samp_width , rate = vals
    out_fids = [ ]
    for idx,sound in enumerate(data):
        out_fid = out_dir/f'{idx}_{row.filename}'
        out_fids.append(out_fid)
        with wave.open(str(out_fid), 'w') as outfile:
            outfile.setnchannels(chans)
            outfile.setsampwidth(samp_width)
            outfile.setframerate(rate)
            outfile.setnframes(int(len(sound) /  samp_width))
            outfile.writeframes(sound)
    all_out.append(out_fids)


# Building an Artifact of 1 second sound clips broken down into class/category name
This:
> - creates a type which is here a parent directory
> - names an artfact for each class
> - creates a list of artfacts which are then itterated through to add a dirctory

#### Our artifacts are going to follow this pattern

In [78]:
import shutil
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')

esc50_csv = Path('./datasets/ESC-50-master/meta/esc50.csv')
path = Path('ESC-50/')
meta_data = path/'meta_data'
meta_data.mkdir(exist_ok=True)
shutil.copyfile(esc50_csv,meta_data/esc50_csv.name)
esc_artifacts = [pth for pth in path.iterdir()]
for pth in tqdm(esc_artifacts[-2:],**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=pth)
    
run.finish()


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.19it/s]


### adding our one second sound file paths to our data frame

In [79]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
df = pd.read_csv(esc50_csv)
out_esc50_csv = meta_data/esc50_csv.name
sec_files = np.array(all_out).astype(str)
for files in range(sec_files.shape[-1]):
    df[f'{files}_{files+1}_sec']=sec_files[...,files].astype(str)
df.to_csv(out_esc50_csv)
log_wandb_artifact(run=run,path=out_esc50_csv)
df

,filename,fold,target,category,esc10,src_file,take,0_1_sec,1_2_sec,2_3_sec,3_4_sec,4_5_sec,5_6_sec
0,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/0_1-100032-A-0.wav,ESC-50/dog/1_1-100032-A-0.wav,ESC-50/dog/2_1-100032-A-0.wav,ESC-50/dog/3_1-100032-A-0.wav,ESC-50/dog/4_1-100032-A-0.wav,ESC-50/dog/5_1-100032-A-0.wav
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,ESC-50/chirping_birds/0_1-100038-A-14.wav,ESC-50/chirping_birds/1_1-100038-A-14.wav,ESC-50/chirping_birds/2_1-100038-A-14.wav,ESC-50/chirping_birds/3_1-100038-A-14.wav,ESC-50/chirping_birds/4_1-100038-A-14.wav,ESC-50/chirping_birds/5_1-100038-A-14.wav
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,ESC-50/vacuum_cleaner/0_1-100210-A-36.wav,ESC-50/vacuum_cleaner/1_1-100210-A-36.wav,ESC-50/vacuum_cleaner/2_1-100210-A-36.wav,ESC-50/vacuum_cleaner/3_1-100210-A-36.wav,ESC-50/vacuum_cleaner/4_1-100210-A-36.wav,ESC-50/vacuum_cleaner/5_1-100210-A-36.wav
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,ESC-50/vacuum_cleaner/0_1-100210-B-36.wav,ESC-50/vacuum_cleaner/1_1-100210-B-36.wav,ESC-50/vacuum_cleaner/2_1-100210-B-36.wav,ESC-50/vacuum_cleaner/3_1-100210-B-36.wav,ESC-50/vacuum_cleaner/4_1-100210-B-36.wav,ESC-50/vacuum_cleaner/5_1-100210-B-36.wav
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,ESC-50/thunderstorm/0_1-101296-A-19.wav,ESC-50/thunderstorm/1_1-101296-A-19.wav,ESC-50/thunderstorm/2_1-101296-A-19.wav,ESC-50/thunderstorm/3_1-101296-A-19.wav,ESC-50/thunderstorm/4_1-101296-A-19.wav,ESC-50/thunderstorm/5_1-101296-A-19.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5-263831-B-6.wav,5,6,hen,False,263831,B,ESC-50/hen/0_5-263831-B-6.wav,ESC-50/hen/1_5-263831-B-6.wav,ESC-50/hen/2_5-263831-B-6.wav,ESC-50/hen/3_5-263831-B-6.wav,ESC-50/hen/4_5-263831-B-6.wav,ESC-50/hen/5_5-263831-B-6.wav
1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A,ESC-50/vacuum_cleaner/0_5-263902-A-36.wav,ESC-50/vacuum_cleaner/1_5-263902-A-36.wav,ESC-50/vacuum_cleaner/2_5-263902-A-36.wav,ESC-50/vacuum_cleaner/3_5-263902-A-36.wav,ESC-50/vacuum_cleaner/4_5-263902-A-36.wav,ESC-50/vacuum_cleaner/5_5-263902-A-36.wav
1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A,ESC-50/footsteps/0_5-51149-A-25.wav,ESC-50/footsteps/1_5-51149-A-25.wav,ESC-50/footsteps/2_5-51149-A-25.wav,ESC-50/footsteps/3_5-51149-A-25.wav,ESC-50/footsteps/4_5-51149-A-25.wav,ESC-50/footsteps/5_5-51149-A-25.wav
1998,5-61635-A-8.wav,5,8,sheep,False,61635,A,ESC-50/sheep/0_5-61635-A-8.wav,ESC-50/sheep/1_5-61635-A-8.wav,ESC-50/sheep/2_5-61635-A-8.wav,ESC-50/sheep/3_5-61635-A-8.wav,ESC-50/sheep/4_5-61635-A-8.wav,ESC-50/sheep/5_5-61635-A-8.wav


In [80]:
run

In [81]:
meta_data_table = wandb.Table(data=df,columns=list(df.columns))
run.log({'meta_data_without_media':meta_data_table})


# Data table for sound Wandb table with 

> 1. ouriginal sound string
> 2. playable sound file
> 3. 4 * 1 second clips files
> 4  4 * 1 second clips files

# Sound and Spectrogram Table



## Adding Sound and Spectrograms to WandB Table  🔉 🪄 🐝

In [82]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
n_examples = 2
n_samples = 10
s_df = df.sample(n=n_samples, random_state=2)
columns = list(df.columns)
table_with_media = wandb.Table(data=s_df,columns=columns)
for colmn in df.columns[-n_examples:]:
    # create wandb sound objects
    sounds = [wandb.Audio(fid) for fid in tqdm(s_df[colmn].values,**tqdm_args,desc=colmn)]
    # add these as a column to our wandb table
    table_with_media.add_column(name=f'sound_{colmn}',data=sounds)

5_6_sec: 100%|████████████████████████████████| 10/10 [00:00<00:00, 2807.43it/s]


### Adding Spectrograms to wandb Table

In [83]:
for col in df.columns[-n_examples:]:
    # read wavs 
    wavs = map(read_wav,tqdm(s_df[col].values,**tqdm_args,desc=col))
    # generate spectrograms
    spects = map(get_arm_spectrogram,wavs)
    sample_rates = iter(np.full((1,len(df)),1600)[0])
    # get numpy arrays of save .jpg files
    ims = list(map(plot_spectrogram,spects,sample_rates))
    # create a list of wandb images
    spects = list(map(wandb.Image,tqdm(ims)))
    # add these to a table 
    table_with_media.add_column(name=f'image_{col}',data=spects)
run.log({'sound_spectrogram_table':table_with_media})

100%|███████████████████████████████████████████| 10/10 [00:10<00:00,  1.02s/it]


## Stack our pandas df by 1 second Files 🧱

In [84]:
rp_df = pd.DataFrame(np.repeat(df.values, 6, axis=0))
rp_df.columns=df.columns
a,b,c,d,e,f = np.array([df[col].values for col in df.columns[-6:]])
all_files =np.vstack((a,b,c,d,e,f)).reshape((-1,),order='F')
rp_df = rp_df[df.columns[:-6]]
rp_df['all_files']=all_files
df = rp_df  
df.to_csv(out_esc50_csv)
log_wandb_artifact(run,out_esc50_csv)
df

,filename,fold,target,category,esc10,src_file,take,all_files
0,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/0_1-100032-A-0.wav
1,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/1_1-100032-A-0.wav
2,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/2_1-100032-A-0.wav
3,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/3_1-100032-A-0.wav
4,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/4_1-100032-A-0.wav
...,...,...,...,...,...,...,...,...
11995,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/1_5-9032-A-0.wav
11996,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/2_5-9032-A-0.wav
11997,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/3_5-9032-A-0.wav
11998,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/4_5-9032-A-0.wav


### Define Test,Train, Val using pandas & add sets column to our dataset

In [86]:
sets = { }
train_val = df.sample(frac=0.9,random_state=42)
sets['test']=df.drop(train_val.index)
sets['train']=train_val.sample(frac=0.9,random_state=42)
sets['val']=train_val.drop(sets['train'].index)
for name,subset in sets.items():
        subset = subset.reset_index(drop=True)
        subset['set']=[name for va in range(len(subset))]
        sets[name]=subset
df.groupby("target", group_keys=False).apply(lambda x: x)
df = pd.concat(sets.values()).reset_index(drop=True)
df.to_csv(out_esc50_csv)
log_wandb_artifact(run,out_esc50_csv)

In [92]:
run

# remove silence by randomly sampling from non silent

In [88]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
is_sound = [not np.std(read_wav(x)) < 0.1 for x in df.all_files.values]
df['is_sound']=is_sound
df.to_csv(out_esc50_csv)
log_wandb_artifact(run,out_esc50_csv)

In [89]:
silence = df[df.is_sound==False]
final_meta_table = wandb.Table(data=silence,columns=list(df.columns))
wandb_audio = list(map(wandb.Audio,silence.all_files.values))
final_meta_table.add_column(name='audio',data=wandb_audio)
run.log({'silence_tabel':final_meta_table})

In [93]:
run

# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [91]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


In [95]:
for pth in df[df.is_sound==False].all_files.values:
    path = Path(pth)
    try:
        path.unlink()
    except FileNotFoundError:
        print('file alredady deleted')
for path in tqdm(esc_artifacts,**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=path)

100%|███████████████████████████████████████████| 51/51 [00:16<00:00,  3.09it/s]


# Save 3D arrays of Spectrograms using numpy

In [96]:
def save_npz(x,y,name,augmenter=None):
    wavs = list(map(read_wav,tqdm(x,f'creating {name}')))
    if augmenter:
        samples = np.full((1,len(y)), 16000)[0]
        wavs = list(map(augmenter,wavs,tqdm(samples,f'augmenting {name}')))
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} augspects'))))
    else:
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} spects'))))
    for array in x_data:
        assert array.shape==(682, 257)
    np.savez(f'{name}.npz',x_data=x_data,y_data=y)

## Loop over files and save as npz, this is our pre-processed training data.

In [97]:
contains_sound = df[df.is_sound==True]
subset =  contains_sound[contains_sound.set=='train']


In [98]:
pre_process=True
contains_sound = df[df.is_sound==True]
if pre_process:
    for name in df.set.unique():
        subset =  contains_sound[contains_sound.set==name]
        x_data = subset['all_files'].values
        y_data = subset['target'].values
        save_npz(x_data,y_data,f'processed/{name}')
        if name=='train':
            print('augmenting')
            save_npz(x_data[:len(x_data)//5],y_data[:len(y_data)//5],f'processed/{name}_aug',augmenter=augmenter)

creating processed/train spects: 100%|█████| 9194/9194 [01:24<00:00, 108.78it/s]


augmenting


augmenting processed/train_aug: 100%|██████▉| 1837/1838 [03:45<00:00,  8.16it/s]
creating processed/train_aug augspects: 100%|█| 1838/1838 [00:15<00:00, 120.38it
creating processed/val spects: 100%|███████| 1030/1030 [00:08<00:00, 119.39it/s]


# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [99]:
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


### To finish our data run

In [100]:
run.finish()
run

# Part 2. Training

### Training load training dataset which has already been pre processed

In [1]:
import tensorflow as tf
import wandb
from pathlib import Path
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import numpy as np
import sounddevice as sd
import wave
from pvrecorder import PvRecorder
import time 
from utils.data_processing import log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect
from utils.create_sweep import create_sweep

In [2]:
with np.load('processed/train.npz',allow_pickle=True) as data:
    train_x = data['x_data'].astype(np.float32)
    train_y = data['y_data'].astype(np.uint8)
with np.load('processed/train_aug.npz',allow_pickle=True) as data:
    aug_x = data['x_data'].astype(np.float32)
    aug_y = data['y_data'].astype(np.uint8)
with np.load('processed/test.npz',allow_pickle=True) as data:
    test_x = data['x_data'].astype(np.float32)
    test_y = data['y_data'].astype(np.uint8)
with np.load('processed/val.npz',allow_pickle=True) as data:
    val_x = data['x_data'].astype(np.float32)
    val_y = data['y_data'].astype(np.uint8)
    

In [3]:
type(train_x)

numpy.ndarray

## create TensoFlow Dataset from numpy arrays of spectrograms and target variable

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_x, tf.float32), train_y))
train_aug_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(aug_x, tf.float32), aug_y))
train_dataset = train_dataset.concatenate(train_aug_dataset)
val_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(val_x, tf.float32),val_y))
test_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(test_x,tf.float32),test_y))

train_ds = train_dataset.cache().shuffle(100, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [7]:
x, y = next(iter(train_ds))
x[0],y[0]

2023-01-30 20:42:45.077465: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(<tf.Tensor: shape=(682, 257), dtype=float32, numpy=
 array([[ 7.65625 ,  6.71875 , 22.296875, ...,  1.875   ,  2.765625,
          3.328125],
        [12.28125 , 10.046875, 21.640625, ...,  2.921875,  3.046875,
          2.5625  ],
        [ 9.      , 19.953125, 30.546875, ...,  2.75    ,  3.03125 ,
          0.796875],
        ...,
        [ 8.171875, 10.484375, 25.4375  , ...,  1.125   ,  0.875   ,
          0.421875],
        [ 1.765625, 18.71875 , 24.359375, ...,  1.25    ,  1.890625,
          0.703125],
        [ 6.4375  , 25.96875 , 31.125   , ...,  2.40625 ,  3.171875,
          2.390625]], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=28>)

## Test that this dataset is loading okay

In [8]:
x,y = next(iter(train_dataset.take(1)))
#print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
#print(input_shape)

In [9]:
print(x)

tf.Tensor(
[[  4.328125   5.046875   5.609375 ...   1.609375   1.203125   0.59375 ]
 [  8.015625   2.5        7.03125  ...   1.59375    2.46875    1.859375]
 [ 15.921875   9.109375  15.5625   ...   2.015625   3.5625     3.5625  ]
 ...
 [ 88.390625  59.9375    12.6875   ...   2.75       2.6875     0.8125  ]
 [100.078125  61.8125    14.1875   ...   2.71875    3.125      0.6875  ]
 [ 88.875     63.03125    5.09375  ...   3.         3.0625     1.84375 ]], shape=(682, 257), dtype=float32)


In [10]:
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_dataset.map(lambda x, y: tf.reshape(x, input_shape)))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-30 20:42:54.297941: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# we have made an application and embeded wandb in the recording process
this is to allow us to :
> - capture new data from you device 🔊
> - Version data using wandb Atefact 🏺
> - Define your wakework 📢

In [11]:
!tree data -d


data
├── background
├── no
└── yes

3 directories


# Create a new training run for our training 🏃🏻‍♀️ ♀️ 

In [12]:
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='training')

wandb: Currently logged in as: frds (tiny-ml). Use `wandb login --relogin` to force relogin


## Define our model 🏗️

In [13]:
baseline_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])

In [14]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 normalization (Normalizatio  (None, 32, 32, 1)        3         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 6, 6, 8)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 288)               0         
                                                                 
 dropout (Dropout)           (None, 288)               0

In [15]:
METRICS = ["accuracy",]
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=METRICS,
)

In [16]:
def scheduler(epoch, lr):
    ''' a function to increase lr at start of trining
    '''
    if epoch < 10:
        return lr
    else:
        # add somthing like np.linespace([0,-0.1])
        return lr * tf.math.exp(-0.1)

In [17]:
import os
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
# Initialize a new W&B run 
checkpoint_path = "training_1/"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights

In [18]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [19]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler)
,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

In [20]:
run

In [21]:
EPOCHS = 250
history = baseline_model.fit(
     train_ds, 
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=val_ds
)

Epoch 1/250
329/345 [===========================>..] - ETA: 0s - loss: 3.5054 - accuracy: 0.0954
Epoch 1: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets
wandb: Adding directory to artifact (./training_1)... Done. 0.0s


345/345 [==============================] - 3s 7ms/step - loss: 3.5017 - accuracy: 0.0956 - val_loss: 3.2672 - val_accuracy: 0.1379 - lr: 0.0030
Epoch 2/250
340/345 [============================>.] - ETA: 0s - loss: 3.1688 - accuracy: 0.1669
Epoch 2: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


...

INFO:tensorflow:Assets written to: training_1/assets
wandb: Adding directory to artifact (./training_1)... Done. 0.0s


345/345 [==============================] - 2s 6ms/step - loss: 2.4894 - accuracy: 0.3108 - val_loss: 3.0803 - val_accuracy: 0.2252 - lr: 2.2282e-04
Epoch 37/250
334/345 [============================>.] - ETA: 0s - loss: 2.4776 - accuracy: 0.3201
Epoch 37: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets
wandb: Adding directory to artifact (./training_1)... Done. 0.0s


345/345 [==============================] - 2s 6ms/step - loss: 2.4760 - accuracy: 0.3194 - val_loss: 3.0774 - val_accuracy: 0.2155 - lr: 2.0162e-04


In [22]:
run.finish()

epoch/accuracy,▁▃▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇███████████
epoch/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,██████████▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,██████████▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/val_accuracy,▁▃▃▅▅▅▅▄▅▅▆▆▆▆▆▆▇▇▇▇▆▇▇▆▇▇▇▇▇▇▇▇█▇██▇
epoch/val_loss,█▅▄▄▂▂▂▂▂▂▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂
epoch/accuracy,0.31943
epoch/epoch,36
epoch/learning_rate,0.0002
epoch/loss,2.47604


## WandB Sweep 🧹

In [35]:
%%writefile wake_word_sweep_mp.py
import os
from wandb.keras import WandbMetricsLogger
from tensorflow.keras.optimizers import Adam, Adamax, Nadam
import wandb
import tensorflow as tf
import numpy as np
from utils.create_sweep import create_sweep
import multiprocessing as mp

def sub_sweep(swp_id):
    def pre_train(config = None):

        run = wandb.init(config=config)

        EPOCHS = run.config.epochs or 100
        LEARNING_RATE = run.config.learning_rate or 0.005
        KERNEL_SIZE = run.config.kernel_size
        DROPOUT_P = run.config.dropout
        ACTIVATION = run.config.activation
        BATCH_SIZE = run.config.batch_size
        BETA_1 = run.config.beta_1
        BETA_2 = run.config.beta_2
        EPSILON = run.config.epsilon

        with np.load('processed/train.npz',allow_pickle=True) as data:
            train_x = data['x_data'].astype(np.float32)
            train_y = data['y_data'].astype(np.uint8)
        with np.load('processed/train_aug.npz',allow_pickle=True) as data:
            aug_x = data['x_data'].astype(np.float32)
            aug_y = data['y_data'].astype(np.uint8)
        with np.load('processed/val.npz',allow_pickle=True) as data:
            val_x = data['x_data'].astype(np.float32)
            val_y = data['y_data'].astype(np.uint8)

        train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_x, tf.float32), train_y))
        train_aug_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(aug_x, tf.float32), aug_y))
        train_dataset = train_dataset.concatenate(train_aug_dataset)

        val_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(val_x, tf.float32),val_y))

        train_ds = train_dataset.cache().shuffle(1000, seed=42).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        val_ds = val_dataset.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

        norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
        x,y = next(iter(train_dataset.take(1)))
        #print(f' target = {y}, \n spectrogram = \n {x}')
        input_shape = tf.expand_dims(x, axis=-1).shape
        #print(input_shape)
        norm_layer.adapt(train_dataset.map(lambda x, y: tf.reshape(x, input_shape)))

        # Initialize a new W&B run 
        checkpoint_path = "training_1/"
        checkpoint_dir = os.path.dirname(checkpoint_path)
        # Create a callback that saves the model's weights
        baseline_model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=input_shape),
            tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
            norm_layer,
            tf.keras.layers.Conv2D(8, kernel_size=(KERNEL_SIZE,KERNEL_SIZE), strides=(2, 2), activation=ACTIVATION),
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(DROPOUT_P),
            tf.keras.layers.Dense(50, activation='softmax')
        ])

        if run.config.optimizer == "adam":
            opt = Adam(
                learning_rate=LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON
                )
        elif run.config.optimizer == "adamax":
            opt = Adamax(
                learning_rate=LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON
            )
        elif run.config.optimizer == "nadam":
            opt = Nadam(
                learning_rate=LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON
            )

        METRICS = ["accuracy",]
        baseline_model.compile(
            optimizer=opt,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=METRICS
        )
        def scheduler(epoch, lr):
            ''' a function to increase lr at start of trining
            '''
            if epoch < 10:
                return lr
            else:
                # add somthing like np.linespace([0,-0.1])
                return lr * tf.math.exp(-0.1)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        verbose=1
        )
        callbacks = [
            tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
            tf.keras.callbacks.LearningRateScheduler(scheduler)
            ,cp_callback,WandbMetricsLogger()]
        history = baseline_model.fit(
             train_ds, 
            epochs=EPOCHS,
            callbacks=callbacks,
            validation_data=val_ds
        )
        run.finish()
wandb.agent(swp_id, function=pre_train,count=10)

    

Overwriting wake_word_sweep_mp.py


In [31]:
from wake_word_sweep_mp import main


In [33]:
main('sweep')

Create sweep with ID: jg8rqet3
Sweep URL: https://wandb.ai/tiny-ml/wake_word_detection/sweeps/jg8rqet3


wandb: Agent Starting Run: m0i6gsnb with config:
wandb: 	activation: gelu
wandb: 	batch_size: 40
wandb: 	beta_1: 0.9
wandb: 	beta_2: 0.987
wandb: 	dropout: 0.04
wandb: 	epochs: 100
wandb: 	epsilon: 1e-07
wandb: 	kernel_size: 9
wandb: 	learning_rate: 0.0007
wandb: 	optimizer: adam
wandb: Agent Starting Run: m0i2hwdh with config:
wandb: 	activation: selu
wandb: 	batch_size: 40
wandb: 	beta_1: 0.5625
wandb: 	beta_2: 0.987
wandb: 	dropout: 0.05
wandb: 	epochs: 100
wandb: 	epsilon: 1e-07
wandb: 	kernel_size: 11
wandb: 	learning_rate: 0.00030000000000000003
wandb: 	optimizer: adam
wandb: Currently logged in as: frds (tiny-ml). Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: frds (tiny-ml). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.9
wandb: Run data is saved locally in /Users/fridadesigley/pico/tiny-ml/wandb/run-20230130_204715-m0i2hwdh
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run helpful-sweep-2


Epoch 1/100
Epoch 1/100
274/276 [============================>.] - ETA: 0s - loss: 3.8460 - accuracy: 0.0423
Epoch 1: saving model to training_1/

Epoch 1: saving model to training_1/
276/276 [==============================] - 19s 65ms/step - loss: 3.6830 - accuracy: 0.0624 - val_loss: 3.5095 - val_accuracy: 0.0903 - lr: 7.0000e-04
Epoch 2/100
  2/276 [..............................] - ETA: 29s - loss: 3.4483 - accuracy: 0.0875

2023-01-30 20:49:25.099924: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at save_restore_v2_ops.cc:286 : NOT_FOUND: training_1/_temp/part-00000-of-00001.data-00000-of-00001; No such file or directory
wandb: Waiting for W&B process to finish... (failed 1). Press Control-C to abort syncing.


 15/276 [>.............................] - ETA: 34s - loss: 3.4304 - accuracy: 0.1000

...

133/276 [=============>................] - ETA: 20s - loss: 2.5281 - accuracy: 0.3126
Epoch 1: saving model to training_1/
92/92 [==============================] - 24s 251ms/step - loss: 4.0199 - accuracy: 0.0220 - val_loss: 4.1368 - val_accuracy: 0.0223 - lr: 9.0000e-04
Epoch 2/100
35/92 [==========>...................] - ETA: 26s - loss: 4.2708 - accuracy: 0.0143
Epoch 31: saving model to training_1/
126/276 [============>.................] - ETA: 20s - loss: 2.5324 - accuracy: 0.3183
Epoch 2: saving model to training_1/
92/92 [==============================] - 34s 372ms/step - loss: 4.4499 - accuracy: 0.0151 - val_loss: 4.6289 - val_accuracy: 0.0194 - lr: 9.0000e-04
Epoch 3/100
41/92 [============>.................] - ETA: 23s - loss: 4.8811 - accuracy: 0.0159
Epoch 32: saving model to training_1/
276/276 [==============================] - 37s 134ms/step - loss: 2.5797 - accuracy: 0.3057 - val_loss: 2.9618 - val_accuracy: 0.2126 - lr: 7.7562e-05
Epoch 33/100
124/276 [============>..

wandb: Ctrl + C detected. Stopping sweep.
wandb: Ctrl + C detected. Stopping sweep.


25/92 [=======>......................] - ETA: 25s - loss: 7.5004 - accuracy: 0.0157

Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:

KeyboardInterrupt

Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/U

204/276 [=====================>........] - ETA: 8s - loss: 2.5460 - accuracy: 0.3146

In [3]:
wandb.agent(sweep_id=sweep,function=pre_train)

wandb: Agent Starting Run: d2ithiyt with config:
wandb: 	activation: tanh
wandb: 	batch_size: 104
wandb: 	beta_1: 0.9
wandb: 	beta_2: 0.9855
wandb: 	dropout: 0.13
wandb: 	epochs: 100
wandb: 	epsilon: 7.5e-08
wandb: 	kernel_size: 9
wandb: 	learning_rate: 0.0006000000000000001
wandb: 	optimizer: adamax
wandb: Currently logged in as: frds (tiny-ml). Use `wandb login --relogin` to force relogin


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-30 14:25:41.812236: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100
106/107 [============================>.] - ETA: 0s - loss: 3.8661 - accuracy: 0.0346
Epoch 1: saving model to training_1/
107/107 [==============================] - 3s 26ms/step - loss: 3.8660 - accuracy: 0.0345 - val_loss: 3.7551 - val_accuracy: 0.0524 - lr: 6.0000e-04
Epoch 2/100
101/107 [===========================>..] - ETA: 0s - loss: 3.7217 - accuracy: 0.0605
Epoch 2: saving model to training_1/
107/107 [==============================] - 1s 10ms/step - loss: 3.7199 - accuracy: 0.0615 - val_loss: 3.6813 - val_accuracy: 0.0709 - lr: 6.0000e-04
Epoch 3/100
101/107 [===========================>..] - ETA: 0s - loss: 3.6553 - accuracy: 0.0745
Epoch 3: saving model to training_1/
107/107 [==============================] - 1s 9ms/step - loss: 3.6584 - accuracy: 0.0741 - val_loss: 3.6276 - val_accuracy: 0.0816 - lr: 6.0000e-04
Epoch 4/100
101/107 [===========================>..] - ETA: 0s - loss: 3.5978 - accuracy: 0.0904
Epoch 4: saving model to training_1/
107/107 [=========

Epoch 30/100
102/107 [===========================>..] - ETA: 0s - loss: 3.1240 - accuracy: 0.1881
Epoch 30: saving model to training_1/
107/107 [==============================] - 1s 9ms/step - loss: 3.1293 - accuracy: 0.1868 - val_loss: 3.1815 - val_accuracy: 0.1806 - lr: 8.1201e-05
Epoch 31/100
105/107 [============================>.] - ETA: 0s - loss: 3.1248 - accuracy: 0.1925
Epoch 31: saving model to training_1/
107/107 [==============================] - 1s 9ms/step - loss: 3.1258 - accuracy: 0.1922 - val_loss: 3.1798 - val_accuracy: 0.1816 - lr: 7.3474e-05
Epoch 32/100
102/107 [===========================>..] - ETA: 0s - loss: 3.1142 - accuracy: 0.1926
Epoch 32: saving model to training_1/
107/107 [==============================] - 1s 9ms/step - loss: 3.1184 - accuracy: 0.1910 - val_loss: 3.1788 - val_accuracy: 0.1825 - lr: 6.6482e-05
Epoch 33/100
105/107 [============================>.] - ETA: 0s - loss: 3.1176 - accuracy: 0.1889
Epoch 33: saving model to training_1/
107/107 [===

...

epoch/accuracy,▁▂▄▅▆▇██▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,██████████▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁
epoch/loss,▅▃▂▁▁▁▁▁▂▂▃▃▄▄▅▅▅▆▆▆▇▇▇▇▇▇████
epoch/lr,██████████▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁
epoch/val_accuracy,▁▁▃▃▅█▇███▇▇▆▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆
epoch/val_loss,▃▂▁▁▁▁▁▂▂▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇█████
epoch/accuracy,0.04288
epoch/epoch,29
epoch/learning_rate,1e-05
epoch/loss,4.16764


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 0xvabav0 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 40
wandb: 	beta_1: 1.125
wandb: 	beta_2: 0.984
wandb: 	dropout: 0.28
wandb: 	epochs: 100
wandb: 	epsilon: 1.125e-07
wandb: 	kernel_size: 11
wandb: 	learning_rate: 0.0008
wandb: 	optimizer: adamax


Epoch 1/100
268/276 [============================>.] - ETA: 0s - loss: 4.1061 - accuracy: 0.0207
Epoch 1: saving model to training_1/
276/276 [==============================] - 4s 14ms/step - loss: 4.1146 - accuracy: 0.0204 - val_loss: 4.3465 - val_accuracy: 0.0165 - lr: 8.0000e-04
Epoch 2/100
270/276 [============================>.] - ETA: 0s - loss: 4.6775 - accuracy: 0.0197
Epoch 2: saving model to training_1/
276/276 [==============================] - 1s 5ms/step - loss: 4.6809 - accuracy: 0.0197 - val_loss: 4.8342 - val_accuracy: 0.0194 - lr: 8.0000e-04
Epoch 3/100
268/276 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0229
Epoch 3: saving model to training_1/
276/276 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.0229 - val_loss: nan - val_accuracy: 0.0262 - lr: 8.0000e-04
Epoch 4/100
270/276 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0189
Epoch 4: saving model to training_1/
276/276 [======================

wandb: Ctrl + C detected. Stopping sweep.


274/276 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0187
Epoch 8: saving model to training_1/
276/276 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.0187 - val_loss: nan - val_accuracy: 0.0262 - lr: 8.0000e-04
Epoch 9/100
274/276 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0187
Epoch 9: saving model to training_1/
276/276 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.0187 - val_loss: nan - val_accuracy: 0.0262 - lr: 8.0000e-04
Epoch 10/100
270/276 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0184
Epoch 10: saving model to training_1/
276/276 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.0187 - val_loss: nan - val_accuracy: 0.0262 - lr: 8.0000e-04
Epoch 11/100
140/276 [==============>...............] - ETA: 0s - loss: nan - accuracy: 0.0168

### Evaluate our locally saved model

In [122]:
latest = tf.train.latest_checkpoint('training_1')
baseline_model.load_weights(latest)

2023-01-30 13:34:28.500761: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open training_1/.: FAILED_PRECONDITION: training_1; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


### Evaluate our Pre-trained Model

In [123]:
baseline_model.evaluate(test_ds, verbose=2)

36/36 - 1s - loss: 2.9813 - accuracy: 0.2124 - 708ms/epoch - 20ms/step


[2.9812943935394287, 0.21235857903957367]

### Dowload model artifact/ our registered model  

In [125]:
api = wandb.Api()
artifact = api.artifact(f'tiny-ml/wake_word_detection/run_{run.id}_model:latest', type='model')
print(artifact.digest,artifact.aliases)
file = artifact.download()  

1bbc16d45ac0c40992e7cbcea9edb31f ['latest', 'epoch_29']


### Load our pre trained tiny model from wandb artifact

In [132]:
baseline_model.load_weights(f'artifacts/run_ex78039g_model:v50/cp.ckpt')
baseline_model.evaluate(test_ds, verbose=2)

36/36 - 0s - loss: 2.4910 - accuracy: 0.3246 - 86ms/epoch - 2ms/step


[2.4910478591918945, 0.32463011145591736]

In [133]:
sample_rate = 16000
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
path = Path('./data/') 
paths = {pth.name:list(map(str,filter(lambda x:x.name != '.gitkeep',pth.iterdir()))) 
         for pth in path.iterdir() if pth.name in ['background','yes']}
paths = {k:np.stack(list(filter(lambda x:len(x)==16384,map(read_wav,tqdm(v))))) 
         for (k,v) in tqdm(paths.items())}

for k,v in paths.items():
    filler_x = [augmenter(x,sample_rate) 
                for x in v[np.random.choice(v.shape[0], 500-len(v), replace=True)]]
    filler_x = np.stack(list(map(lambda x:x[:16384], tqdm(filler_x))))
    paths[k]=np.array(list(map(get_arm_spectrogram,tqdm(np.vstack([v,filler_x]))))).astype(np.float32)


100%|█████████████████████████████████████████| 90/90 [00:00<00:00, 2587.09it/s]

100%|████████████████████████████████████████| 500/500 [00:01<00:00, 389.99it/s]


In [134]:
paths.values()

dict_values([array([[[1.07015625e+02, 6.92031250e+01, 1.55312500e+01, ...,
         1.71875000e-01, 1.56250000e-01, 1.56250000e-01],
        [1.22921875e+02, 6.72656250e+01, 4.59375000e+00, ...,
         6.25000000e-02, 6.25000000e-02, 7.81250000e-02],
        [1.27843750e+02, 6.41562500e+01, 1.71875000e-01, ...,
         0.00000000e+00, 0.00000000e+00, 1.56250000e-02],
        ...,
        [1.28031250e+02, 6.39843750e+01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.28031250e+02, 6.39843750e+01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.28031250e+02, 6.39843750e+01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.07015625e+02, 6.92031250e+01, 1.55312500e+01, ...,
         1.71875000e-01, 1.56250000e-01, 1.56250000e-01],
        [1.22921875e+02, 6.72656250e+01, 4.59375000e+00, ...,
         6.25000000e-02, 6.25000000e-02, 7.81250000e-02],
        [1.2

In [135]:
y = np.concatenate([np.zeros((1,500)).astype(np.uint16)[0], np.ones((1,500)).astype(np.uint16)[0]])
x = np.vstack(list(paths.values()))

In [136]:
train_wake_data = tf.data.Dataset.from_tensor_slices((tf.cast(x,tf.float32), y))
train_wake_loader = train_wake_data.cache().shuffle(2000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)

In [137]:
x,y = next(iter(train_wake_data.take(1)))
print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_wake_data.map(lambda x, y: tf.reshape(x, input_shape)))


 target = 0, 
 spectrogram = 
 [[1.07015625e+02 6.92031250e+01 1.55312500e+01 ... 1.71875000e-01
  1.56250000e-01 1.56250000e-01]
 [1.22921875e+02 6.72656250e+01 4.59375000e+00 ... 6.25000000e-02
  6.25000000e-02 7.81250000e-02]
 [1.27843750e+02 6.41562500e+01 1.71875000e-01 ... 0.00000000e+00
  0.00000000e+00 1.56250000e-02]
 ...
 [1.28031250e+02 6.39843750e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.28031250e+02 6.39843750e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.28031250e+02 6.39843750e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [138]:
input_shape = tf.expand_dims(x, axis=-1).shape
print(input_shape)

(249, 257, 1)


In [139]:
def fine_tune(run=None):
    base_model = tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=input_shape),
      tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
      norm_layer,
      tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
      tf.keras.layers.MaxPool2D(pool_size=(2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Dense(50, activation='softmax')
    ])

    # Initialize a new W&B run
    run = wandb.init(config={"bs": 12},entity='tiny-ml',project = 'wake_word_detection', group='training',settings={'quiet':True,'silent':True,'show_warnings':False,'show_info':False})


    EPOCHS = 100
    history = wake_word_model.fit(
        train_wake_loader, 
        epochs=EPOCHS,
        callbacks=callbacks
    )
    model_body = tf.keras.Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
    classifier_head = tf.keras.layers.Dense(1, activation="sigmoid")(model_body.output)
    wake_word_model = tf.keras.Model(model_body.input, classifier_head)

    checkpoint_path = "training_2/"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    print(checkpoint_dir)
    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        verbose=1
    )
    callbacks = [
        tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
        tf.keras.callbacks.LearningRateScheduler(scheduler)
    ,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

    wake_word_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=METRICS,
    )

    EPOCHS = 100
    history = wake_word_model.fit(
        train_wake_loader, 
        epochs=EPOCHS,
        callbacks=callbacks)
    
    

In [141]:

wake_word_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 249, 257, 1)]     0         
                                                                 
 resizing_1 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_1 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 288)               0   

In [142]:
for layer in wake_word_model.layers:
    layer.trainable = False
wake_word_model.layers[-1].trainable = True

In [143]:
# Initialize a new W&B run
run = wandb.init(config={"bs": 12},entity='tiny-ml',project = 'wake_word_detection', group='training',settings={'quiet':True,'silent':True,'show_warnings':False,'show_info':False})



training_2


In [144]:
wake_word_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS,
)

In [145]:
EPOCHS = 100
history = wake_word_model.fit(
    train_wake_loader, 
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
 1/32 [..............................] - ETA: 8s - loss: 0.8022 - accuracy: 0.3438WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy



Epoch 1: saving model to training_2/


INFO:tensorflow:Assets written to: training_2/assets


INFO:tensorflow:Assets written to: training_2/assets


32/32 [==============================] - 1s 32ms/step - loss: 0.6432 - accuracy: 0.6460 - lr: 0.0030
Epoch 2/100
23/32 [====================>.........] - ETA: 0s - loss: 0.4952 - accuracy: 0.7894WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


...

32/32 [==============================] - 1s 25ms/step - loss: 0.3289 - accuracy: 0.8790 - lr: 4.0917e-07
Epoch 100/100
 1/32 [..............................] - ETA: 0s - loss: 0.3597 - accuracy: 0.8438WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy



Epoch 100: saving model to training_2/


INFO:tensorflow:Assets written to: training_2/assets


INFO:tensorflow:Assets written to: training_2/assets


32/32 [==============================] - 1s 31ms/step - loss: 0.3408 - accuracy: 0.8750 - lr: 3.7023e-07


In [190]:
for layer in wake_word_model.layers:
    layer.trainable = True

In [191]:

import tensorflow_model_optimization as tfmot

def apply_qat_to_dense_and_cnn(layer):
  if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    wake_word_model,
    clone_function=apply_qat_to_dense_and_cnn,
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 249, 257, 1)]     0         
                                                                 
 resizing_7 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_7 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 quant_conv2d_7 (QuantizeWra  (None, 13, 13, 8)        539       
 pperV2)                                                         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                           

In [197]:
quant_aware_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=METRICS,
)

EPOCHS=100
quant_aware_history = quant_aware_model.fit(
    train_wake_loader,  
    epochs=EPOCHS
)

Epoch 1/100
32/32 [==============================] - 1s 3ms/step - loss: 0.3295 - accuracy: 0.8700
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2932 - accuracy: 0.8800
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3004 - accuracy: 0.8810
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2875 - accuracy: 0.8880
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3177 - accuracy: 0.8870
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2913 - accuracy: 0.8890
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.2545 - accuracy: 0.8830
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2657 - accuracy: 0.9040
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2729 - accuracy: 0.8990
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2448 - accuracy: 0.9070
Epoch 11/

32/32 [==============================] - 0s 3ms/step - loss: 0.1619 - accuracy: 0.9320
Epoch 84/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1634 - accuracy: 0.9350
Epoch 85/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1693 - accuracy: 0.9330
Epoch 86/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1794 - accuracy: 0.9300
Epoch 87/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1690 - accuracy: 0.9350
Epoch 88/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1643 - accuracy: 0.9350
Epoch 89/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1597 - accuracy: 0.9320
Epoch 90/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1420 - accuracy: 0.9420
Epoch 91/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1490 - accuracy: 0.9430
Epoch 92/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1544 - accuracy: 0.9390
Epoch 93/100


In [198]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmp4_1be5sa/assets


INFO:tensorflow:Assets written to: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmp4_1be5sa/assets
/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-01-29 22:08:23.786690: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-29 22:08:23.786702: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-29 22:08:23.786789: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmp4_1be5sa
2023-01-29 22:08:23.788279: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-29 22:08:23.788285: I tensorflow/cc/saved_model/reader.cc:130] Readin

In [204]:
tflite_models_dir = Path("tf_lite_model")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"wake_word.tflite"
tflite_model_file.write_bytes(tflite_model_quant)

6016

In [207]:
%%bash
echo "alignas(8) const unsigned char tflite_model[] = {" > tf_lite_model/wake_word.tflite.h
cat tf_lite_model/wake_word.tflite | xxd -i                        >> tf_lite_model/wake_word.tflite.h
echo "};"                                               >> tf_lite_model/wake_word.tflite.h

In [208]:
!ls tf_lite_model/

wake_word.tflite   wake_word.tflite.h
